In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.seurat = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/obj.seurat.rds"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/pairs.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR_250226/BMMC22.merge.250317/BMMC22.merge.250317/"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import seurat object

In [5]:
obj.seurat = readRDS(path.seurat)

Import candidate E-G pairs

In [6]:
pairs.E2G = readRDS(path.pairs.E2G)

Create an ArchRProject

In [7]:
str(Fragments(obj.seurat))

List of 13
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "86350e844e0a3a1991e9c650ccac7f43" "555000121b665f095f13712669597ac8"
  .. ..@ cells: Named chr [1:6224] "TAGTTGTCACCCTCAC-1" "CTATGGCCATAACGGG-1" "CCGCACACAGGTTAAA-1" "TCATTTGGTAATGGAA-1" ...
  .. .. ..- attr(*, "names")= chr [1:6224] "s1d1_TAGTTGTCACCCTCAC-1" "s1d1_CTATGGCCATAACGGG-1" "s1d1_CCGCACACAGGTTAAA-1" "s1d1_TCATTTGGTAATGGAA-1" ...
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "3ee058ae058031e3f233c68b0db9e6b9" "1039e761a9da92608ebabb952d11b3c8"
  .. ..@ cells: Named chr [1:6740] "CTACCGGCAGTTGCGT-1" "GATGCAGCAGCAACAG-1" "TGAGTGAAGAGGAAGG-1" "CCGCTA

In [8]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(Fragments(obj.seurat)[[1]]@path,
                 Fragments(obj.seurat)[[2]]@path,
                 Fragments(obj.seurat)[[3]]@path,
                 Fragments(obj.seurat)[[4]]@path,
                 Fragments(obj.seurat)[[5]]@path,
                 Fragments(obj.seurat)[[6]]@path,
                 Fragments(obj.seurat)[[7]]@path,
                 Fragments(obj.seurat)[[8]]@path,
                 Fragments(obj.seurat)[[9]]@path,
                 Fragments(obj.seurat)[[10]]@path,
                 Fragments(obj.seurat)[[11]]@path,
                 Fragments(obj.seurat)[[12]]@path,
                 Fragments(obj.seurat)[[13]]@path),
  sampleNames = c("s1d1",
                  "s1d2",
                  "s1d3",
                  "s2d1",
                  "s2d4",
                  "s2d5",
                  "s3d10",
                  "s3d3",
                  "s3d6",
                  "s3d7",
                  "s4d1",
                  "s4d8",
                  "s4d9"),
  validBarcodes = list(s1d1 = unname(Fragments(obj.seurat)[[1]]@cells),
                       s1d2 = unname(Fragments(obj.seurat)[[2]]@cells),
                       s1d3 = unname(Fragments(obj.seurat)[[3]]@cells),
                       s2d1 = unname(Fragments(obj.seurat)[[4]]@cells),
                       s2d4 = unname(Fragments(obj.seurat)[[5]]@cells),
                       s2d5 = unname(Fragments(obj.seurat)[[6]]@cells),
                       s3d10 = unname(Fragments(obj.seurat)[[7]]@cells),
                       s3d3 = unname(Fragments(obj.seurat)[[8]]@cells),
                       s3d6 = unname(Fragments(obj.seurat)[[9]]@cells),
                       s3d7 = unname(Fragments(obj.seurat)[[10]]@cells),
                       s4d1 = unname(Fragments(obj.seurat)[[11]]@cells),
                       s4d8 = unname(Fragments(obj.seurat)[[12]]@cells),
                       s4d9 = unname(Fragments(obj.seurat)[[13]]@cells)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-183fb6441ec1b3-Date-2025-03-17_Time-17-34-20.284561.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2025-03-17 17:34:20.655211 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-183fb6441ec1b3-Date-2025-03-17_Time-17-34-20.284561.log



In [9]:
ArrowFiles

[1] "s3d10.arrow" "s3d3.arrow"  "s3d7.arrow"  "s4d1.arrow"  "s1d1.arrow" 
 [6] "s4d8.arrow"  "s2d4.arrow"  "s2d1.arrow"  "s2d5.arrow"  "s1d2.arrow" 
[11] "s1d3.arrow"  "s4d9.arrow"  "s3d6.arrow"

In [10]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

In [11]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...



Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 


Getting Cell Metadata...



Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__           

In [12]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR_250226/BMMC22.merge.250317/BMMC22.merge.250317/ArchR 
samples(13): s3d10 s3d3 ... s4d9 s3d6
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 67013
medianTSS(1): 16.107
medianFrags(1): 9747

In [13]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-183fb62d34030a-Date-2025-03-17_Time-18-15-51.513029.log
If there is an issue, please report to github with logFile!

2025-03-17 18:15:53.578401 : Computing Total Across All Features, 0.019 mins elapsed.

2025-03-17 18:16:00.968106 : Computing Top Features, 0.143 mins elapsed.

###########
2025-03-17 18:16:06.095866 : Running LSI (1 of 2) on Top Features, 0.228 mins elapsed.
###########

2025-03-17 18:16:06.167298 : Sampling Cells (N = 10007) for Estimated LSI, 0.229 mins elapsed.

2025-03-17 18:16:06.172835 : Creating Sampled Partial Matrix, 0.229 mins elapsed.

2025-03-17 18:16:18.980752 : Computing Estimated LSI (projectAll = FALSE), 0.443 mins elapsed.

2025-03-17 18:17:09.384171 : Identifying Clusters, 1.283 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2025-03-17 18:17:25.041169 : Identified 6 Clusters, 1.544 mins elapsed.

2025-03-17 18:17:25.068293 : Saving LSI Iteration, 1


************************************************************
2025-03-17 18:17:42.556368 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-183fb62d34030a-Date-2025-03-17_Time-18-15-51.513029.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2025-03-17 18:17:42.560317 : Creating Cluster Matrix on the total Group Features, 1.836 mins elapsed.

2025-03-17 18:18:04.990031 : Computing Variable Features, 2.21 mins elapsed.

###########
2025-03-17 18:18:05.150654 : Running LSI (2 of 2) on Variable Features, 2.212 mins elapsed.
###########

2025-03-17 18:18:05.194739 : Creating Partial Matrix, 2.213 mins elapsed.

2025-03-17 18:18:32.040475 : Computing LSI, 2.661 mins elapsed.

2025-03-17 18:19:52.699161 : Finished Running IterativeLSI, 4.005 mins elapsed.



In [14]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [15]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-183fb64d68f7b4-Date-2025-03-17_Time-18-19-58.45656.log
If there is an issue, please report to github with logFile!

2025-03-17 18:19:58.750583 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addPeakMatrix-183fb64d68f7b4-Date-2025-03-17_Time-18-19-58.45656.log



In [16]:
paths.h5 = c("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor2/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor4/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor5/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor10/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor6/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor7/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor8/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor9/outs/raw_feature_bc_matrix.h5")
names.sample = c("s1d1",
                 "s1d2",
                 "s1d3",
                 "s2d1",
                 "s2d4",
                 "s2d5",
                 "s3d10",
                 "s3d3",
                 "s3d6",
                 "s3d7",
                 "s4d1",
                 "s4d8",
                 "s4d9")
list.seRNA = lapply(1:length(names.sample),function(x){
    import10xFeatureMatrix(paths.h5[x],
                           names.sample[x])
})

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.


In [17]:
list.seRNA

[[1]]
class: RangedSummarizedExperiment 
dim: 61218 727198 
metadata(0):
assays(1): counts
rownames(61218): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(727198): s1d1#AAACAGCCAAACAACA-1 s1d1#AAACAGCCAAACATAG-1 ...
  s1d1#TTTGTTGGTTTGTCTA-1 s1d1#TTTGTTGGTTTGTGGA-1
colData names(0):

[[2]]
class: RangedSummarizedExperiment 
dim: 61218 726937 
metadata(0):
assays(1): counts
rownames(61218): DDX11L2 DDX11L1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(726937): s1d2#AAACAGCCAAACAACA-1 s1d2#AAACAGCCAAACATAG-1 ...
  s1d2#TTTGTTGGTTTGTGGA-1 s1d2#TTTGTTGGTTTGTTGC-1
colData names(0):

[[3]]
class: RangedSummarizedExperiment 
dim: 61218 724480 
metadata(0):
assays(1): counts
rownames(61218): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(724480): s1d3#AAACAGCCAAACAACA-1 s1d3#AAACAGCCAAACATAG-1 ...
  s1d3#TTTGTTGGTTTGTG

In [18]:
id.intersect = rowData(list.seRNA[[1]])[,"id"]
length(id.intersect)
for(n.tmp in 2:13){
    id.intersect = intersect(id.intersect,rowData(list.seRNA[[n.tmp]])[,"id"])
}
length(id.intersect)

[1] 61218

[1] 61165

In [19]:
seRNA = list.seRNA[[1]][rowData(list.seRNA[[1]])[,"id"] %in% id.intersect]
for(n.tmp in 2:13){
    seRNA = cbind(seRNA,list.seRNA[[n.tmp]][rowData(list.seRNA[[n.tmp]])[,"id"] %in% id.intersect])
}
seRNA

class: RangedSummarizedExperiment 
dim: 61165 9222766 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(9222766): s1d1#AAACAGCCAAACAACA-1 s1d1#AAACAGCCAAACATAG-1 ...
  s4d9#TTTGTTGGTTTGTGGA-1 s4d9#TTTGTTGGTTTGTTGC-1
colData names(0):

In [20]:
seRNA = seRNA[,rownames(obj.ArchR@cellColData)]

In [21]:
seRNA

class: RangedSummarizedExperiment 
dim: 61165 67013 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(67013): s3d10#GTAGCTGTCAGCAAAG-1 s3d10#AACTAGTGTTACCTGT-1 ...
  s3d6#TCACCGGCAGCCTTGG-1 s3d6#GAACCTGTCGTTACAA-1
colData names(0):

Map gene names

In [22]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [23]:
seRNA.rename = map_gene_names(seRNA,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following object is masked from ‘package:Biobase’:

    combine


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:matrixStats’:

    count


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:plyr’

In [24]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA.rename,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-183fb641efd635-Date-2025-03-17_Time-18-28-36.836904.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2025-03-17 18:28:38.010964 : 

Overlap Per Sample w/ scATAC : s1d1=6149,s1d2=6685,s1d3=4260,s2d1=3966,s2d4=5321,s2d5=4558,s3d10=6724,s3d3=4269,s3d6=1645,s3d7=1627,s4d1=7905,s4d8=9705,s4d9=4199

2025-03-17 18:28:38.117947 : 

2025-03-17 18:28:43.951024 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addGeneExpressionMatrix-183fb641efd635-Date-2025-03-17_Time-18-28-36.836904.log



Run ArchR prediction

In [25]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-183fb61724756-Date-2025-03-17_Time-18-32-48.144047.log
If there is an issue, please report to github with logFile!

2025-03-17 18:32:50.670992 : Getting Available Matrices, 0.042 mins elapsed.

2025-03-17 18:33:07.476118 : Filtered Low Prediction Score Cells (0 of 67013, 0), 0.062 mins elapsed.

2025-03-17 18:33:11.194274 : Computing KNN, 0.124 mins elapsed.

2025-03-17 18:33:11.667298 : Identifying Non-Overlapping KNN pairs, 0.132 mins elapsed.

2025-03-17 18:33:13.868196 : Identified 500 Groupings!, 0.169 mins elapsed.

2025-03-17 18:33:13.96766 : Getting Group RNA Matrix, 0.171 mins elapsed.

2025-03-17 18:34:25.291775 : Getting Group ATAC Matrix, 1.359 mins elapsed.

2025-03-17 18:36:14.262678 : Normalizing Group Matrices, 3.176 mins elapsed.

2025-03-17 18:36:28.574838 : Finding Peak Gene Pairings, 3.414 mins elapsed.

2025-03-17 18:36:30.719868 : Computing Correlations, 3.45 mins elapsed.

2025-03-17 18:37:13.665362 : Completed

In [26]:
execution_time

Time difference of 4.426522 mins

In [27]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [28]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [29]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [30]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 11910478 rows and 7 columns
                                idxATAC    idxRNA Correlation       FDR
                              <integer> <integer>   <numeric> <numeric>
chr1-10010-10536_FAM138A              1         1         NaN       NaN
chr1-96496-96699_FAM138A              2         1         NaN       NaN
chr1-115495-115930_FAM138A            3         1         NaN       NaN
chr1-180470-181899_FAM138A            4         1         NaN       NaN
chr1-184379-184607_FAM138A            5         1         NaN       NaN
...                                 ...       ...         ...       ...
chrX-156008869-156009386_IL9R    540425     20486  -0.0467826  0.532075
chrX-156019809-156020162_IL9R    540426     20486   0.0524075  0.469659
chrX-156025098-156025397_IL9R    540427     20486   0.0828137  0.187923
chrX-156029831-156030380_IL9R    540428     20486   0.0439540  0.563653
chrX-156030497-156030869_IL9R    540429     20486   0.0870693  0.159768
                     

In [31]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [32]:
pairs.E2G.res

GRanges object with 9765178 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100004103-100004301      * |         AGL
        [2]     chr1 100004103-100004301      * |      CDC14A
        [3]     chr1 100004103-100004301      * |         DBT
        [4]     chr1 100004103-100004301      * |       EXTL2
        [5]     chr1 100004103-100004301      * |       FRRS1
        ...      ...                 ...    ... .         ...
  [9765174]     chrX   99980528-99980848      * |      PCDH19
  [9765175]     chrX   99980528-99980848      * |      TSPAN6
  [9765176]     chrX   99980528-99980848      * |        XKRX
  [9765177]     chrX     9999312-9999592      * |       TBL1X
  [9765178]     chrX     9999312-9999592      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

Save results

In [33]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 9765178 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100004103-100004301      * |         AGL
        [2]     chr1 100004103-100004301      * |      CDC14A
        [3]     chr1 100004103-100004301      * |         DBT
        [4]     chr1 100004103-100004301      * |       EXTL2
        [5]     chr1 100004103-100004301      * |       FRRS1
        ...      ...                 ...    ... .         ...
  [9765174]     chrX   99980528-99980848      * |      PCDH19
  [9765175]     chrX   99980528-99980848      * |      TSPAN6
  [9765176]     chrX   99980528-99980848      * |        XKRX
  [9765177]     chrX     9999312-9999592      * |       TBL1X
  [9765178]     chrX     9999312-9999592      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100004103,100004301,AGL,K562,-0.007710668,9.324562e-01,0.03058681,0.78521917
chr1,100004103,100004301,CDC14A,K562,-0.145382884,6.950791e-03,0.03058681,0.97881480
chr1,100004103,100004301,DBT,K562,0.034985805,6.632382e-01,0.03058681,0.63213902
chr1,100004103,100004301,EXTL2,K562,-0.055170442,4.396358e-01,0.03058681,0.51020209
chr1,100004103,100004301,FRRS1,K562,0.108531036,6.157165e-02,0.03058681,0.66259885
chr1,100004103,100004301,MFSD14A,K562,0.034960156,6.635091e-01,0.03058681,0.39387875
chr1,100004103,100004301,PLPPR5,K562,-0.035251693,6.603494e-01,0.03058681,0.33876794
chr1,100004103,100004301,RTCA,K562,0.063551546,3.524594e-01,0.03058681,0.68192912
chr1,100004103,100004301,RTCA-AS1,K562,0.123300150,2.794338e-02,0.03058681,0.46983306


In [34]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4           